In [1]:
import os
import xml.etree.ElementTree as ET

In [2]:
# Adapted from https://github.com/burakuyar/Tools/blob/master/musicxml_player.py

def parse_notes(root):
    """
    Returns a 2D array of [[note_idx,note_duration,note_name]]
    Note name can only be "PitchClass Octave", "PitchClass Octave Accidental" or "Rest"
    If a note_name is not Rest and it has zero duration, that note is an embellishment.
    """
    notes=[]
    for i,note in enumerate(root.findall('part/measure/note')):

        dur=note.find('duration')
        if dur is None:
            dur=0 # Embellishment, Grace Note
        else:
            dur=dur.text

        step=note.find('pitch/step')
        if step is not None:
            step=step.text
            octave=note.find('pitch/octave').text
            acc = note.find('accidental')
            if acc is None:
                n='{}{}'.format(step, octave)
            else:
                n='{}{} {}'.format(step, octave, acc.text)
        else:
            rest = note.find('rest')
            assert rest is not None, "The note doesn't have a pitch and is not a rest!"
            n='Rest'

        note = [i, dur, n]
        notes.append(note)
    return notes

def get_bpm(root):
    return float(root.find('part/measure/direction/sound').attrib['tempo'])

def get_divisions(root):
    return float(root.find('part/measure/attributes/divisions').text)

def find_key_signature_accidentals(root):
    notes = []
    accidentals = []
    for k in root.iter('key'):
        for ks in k.findall('key-step'):
            notes.append(ks.text)
        for ka in k.findall('key-accidental'):
            accidentals.append(ka.text)
    return ['{} {}'.format(n,k) for n,k in zip(notes,accidentals)]

def find_all_accidentals(root):
    # Access accidental names as follows
    accidentals=[]
    for a in root.iter('accidental'):
        #print(a.text)
        accidentals.append(a.text)
    accidentals=set(accidentals)
    #print(accidentals)
    return accidentals
        
#qnotelen = 60000/bpm

# 1) Read an XML file

In [3]:
score_name='hicaz--sarki--aksaksemai--sezdim_dargin--rifat_ayaydin.xml'

data_dir=os.path.join(os.getcwd(), 'data')
score_path=os.path.join(data_dir, score_name)

tree=ET.parse(score_path)
root=tree.getroot()

print(root.tag)

score-partwise


In [5]:
bpm=get_bpm(root)
print(f'BPM: {bpm}')
divs=get_divisions(root)
print('Divisions: {}'.format(divs))
key_signature_accidentals=find_key_signature_accidentals(root)
print('Accidentals in the key signature: {}'.format(key_signature_accidentals))
notes=parse_notes(root)

BPM: 65.0
Divisions: 96.0
Accidentals in the key signature: ['B slash-flat', 'F sharp', 'C sharp']


In [10]:
print('note_idx, note_duration, note_name')
print('='*len('note_idx, note_duration, note_name'))
for n in notes:
    print(n)   

note_idx, note_duration, note_name
[0, '48', 'G5']
[1, '48', 'F5 sharp']
[2, '24', 'G5']
[3, '24', 'E5']
[4, 0, 'F5']
[5, '96', 'E5']
[6, '48', 'E5']
[7, '24', 'F5 sharp']
[8, '24', 'G5']
[9, '24', 'A5']
[10, '24', 'G5']
[11, '24', 'F5']
[12, '24', 'E5']
[13, '48', 'D5']
[14, 0, 'D5']
[15, '48', 'C5 sharp']
[16, '48', 'B4 slash-flat']
[17, '48', 'A4']
[18, '72', 'D5']
[19, '24', 'C5 sharp']
[20, '24', 'C5 sharp']
[21, '24', 'B4 slash-flat']
[22, '24', 'B4 slash-flat']
[23, '24', 'A4']
[24, 0, 'B4']
[25, '144', 'A4']
[26, '48', 'A4']
[27, '48', 'B4 slash-flat']
[28, '48', 'C5 sharp']
[29, '48', 'E5']
[30, '24', 'D5']
[31, '24', 'C5 sharp']
[32, '24', 'C5 sharp']
[33, '24', 'B4 slash-flat']
[34, '24', 'B4 slash-flat']
[35, '24', 'A4']
[36, 0, 'B4']
[37, '72', 'A4']
[38, '24', 'G4']
[39, '12', 'F5']
[40, '12', 'E5']
[41, '12', 'D5']
[42, '12', 'E5']
[43, '96', 'D5']
[44, '48', 'E5']
[45, '48', 'E5']
[46, '48', 'D5']
[47, '96', 'D5']
[48, 0, 'E5']
[49, '72', 'D5']
[50, '24', 'C5 sharp']
[5

I realized that Some of the duration None notes are grace notes. They are embellishments.
left for the player. Marked by smaller font notes on the score. Still have
pitches